In [3]:
!git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null

UsageError: Line magic function `%git` not found.


In [ ]:
import requests

with requests.get("https://case.law/download/bulk_exports/latest/by_jurisdiction/case_text_open/ark/ark_text.zip", stream=True) as r:
        r.raise_for_status()
        with open("corpus.zip", 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                #if chunk: 
                f.write(chunk)

In [ ]:
import zipfile

with zipfile.ZipFile("corpus.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [ ]:
!xz --decompress /content/data/ark_text_20210921/data/data.jsonl.xz

'xz' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [ ]:
import json
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
with open('/content/data/ark_text_20210921/data/data.jsonl', 'r') as json_file:
    json_list = list(json_file)

results = []
for json_str in json_list:
  text = json.loads(json_str)
  if text["casebody"]["data"]["opinions"]:
    results.append(
    text["casebody"]["data"]["opinions"][0]["text"]
    text["court"]["name"]
    text["frontend_url"]
    text["decision_date"]
    text["name_abbreviation"]
    text["casebody"]["data"]["opinions"][0]["author"]
      )

### Pandas

In [ ]:
    df = pd.DataFrame(results, columns=["text", "author", "court", "url", "date", "name"])
    df.head()

Introduction

Voici votre premier mini-projet sur le traitement du langage naturel (NLP). En d'autres termes, nous allons apprendre à la machine à lire ! Tout d'abord, nous allons voir comment faire de l'exploration de texte simple sur un dataset du CAP (arkansas).

Dans ce premier mini-projet, vous apprendrez à :

- accéder au jeu de données et le comprendre
- le convertir en un DataFrame pandas pour une simple exploration de texte.

# Blueprint : Obtenir une vue d'ensemble des données avec Pandas


## Calcul des statistiques sommaires pour les colonnes

Le but n'est pas de vous apprendre les pandas ici, car il existe d'excellents tutoriels. Le but est de vous montrer à quel point Pandas est puissant et rapide.

Dans ipython, après avoir exécuté le script, nous avons un accès interactif à notre objet DataFrame, appelé df.

Commençons par examiner les valeurs possibles pour les étoiles :

In [ ]:
authors = df['author'] 
sorted(authors.unique())

In [ ]:
df['length'] = df['text'].str.len()

df.describe().T

In [ ]:
df[['court', 'author']].describe(include='O').T

## Vérification des données manquantes

In [ ]:
df.isna().sum()

In [ ]:
df['author'].fillna('unkown', inplace=True)

In [ ]:
df[df['author'].str.contains('Bush')]['author'].value_counts()

## Représenter les distributions de valeurs

In [ ]:
df['length'].plot(kind='box', vert=False, figsize=(8, 1))

In [ ]:
df['length'].plot(kind='hist', bins=30, figsize=(8,2))

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 2))
sns.distplot(df['length'], bins=30, kde=True);

## Comparing Value Distributions across Categories


In [ ]:
where = df['country'].isin(['USA', 'FRA', 'GBR', 'CHN', 'RUS'])
g = sns.catplot(data=df[where], x="country", y="length", kind='box')
g.fig.set_size_inches(4, 3) ###
g.fig.set_dpi(100) ###
g = sns.catplot(data=df[where], x="country", y="length", kind='violin')
g.fig.set_size_inches(4, 3) ###
g.fig.set_dpi(100) ###

## Visualizing Developments over Time


In [ ]:
df.groupby('year').size().plot(title="Number of Countries", figsize=(6,2))

In [ ]:
df.groupby('year').agg({'length': 'mean'}) \
  .plot(title="Avg. Speech Length", ylim=(0,30000), figsize=(6,2))

1. Jour 1
    * Variables
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.1%20helloworld.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.2%20%C3%A9viter%20les%20errreurs%20de%20nommage.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.3%20mini-project.ipynb#scrollTo=RPB2xCMdA6lV)
    * Strings
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.1%20concat.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.2%20string_methods.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.3%20mini-project.ipynb)
    * Opérations
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.1%20math.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.2%20bool%C3%A9en.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.3%20mini-project.ipynb)

2. Jour 2
    * Listes
        * [Définition](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.1%20d%C3%A9finition%20liste.ipynb?hl=fr)
        * [Strings et listes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.2%20String%20as%20list%20of%20characters.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.3%20mini-project.ipynb?hl=fr)
    * Fonctions
        * [Définition I](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.1%20d%C3%A9finition%20fonctions.ipynb?hl=fr)
        * [Définition II](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.2%20scope%20et%20fonctions%20imbriqu%C3%A9es.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.3%20mini-project.ipynb?hl=fr) 
    * Librairies
        * [Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.1%20Pandas.ipynb?hl=fr)
        * [Matplotlib](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.2%20Matplotlib.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.3%20mini-project.ipynb?hl=fr) 
3. Jour 3
    * Introduction à la NLP
        * [Charger des un corpus](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.1%20Accessing%20Text.ipynb?hl=fr)
        * [Traitement de texte dans Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.2%20Working%20with%20text%20data%20in%20pandas.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.3%20mini-project.ipynb?hl=fr)
    * Segmentation
        * [Segmentation de tokens](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.1%20Token%20segmentation.ipynb?hl=fr)
        * [Segmentation de phrase](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.2%20Sentence%20segmentation.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.3%20mini-project.ipynb?hl=fr)
    * Nettoyage de texte
        * [Stopwords](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.1%20stopwords.ipynb?hl=fr)
        * [Normalisation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.2%20Normalizing%20Text.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.3%20mini-project.ipynb?hl=fr)
4. Jour 4
    * Apprentissage supervisé
        * [Régression linéaire](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.1%20linear%20regression.ipynb?hl=fr)
        * [Evaluation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.2%20evaluale.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.3%20mini-project.ipynb?hl=fr)
    * Pré-traitement de texte
        * [Featurization de textes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.1%20text%20featurization.ipynb?hl=fr)
        * [Featurization de labels](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.2%20label%20featurization.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.3%20mini-project.ipynb?hl=fr)
    * Classification de texte
        * [EDA](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Apprentissage supervisé textuel](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.3%20mini-project.ipynb?hl=fr)
5. Jour 5
    * [Projet final](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour5/final-project.ipynb?hl=fr)